In [30]:
import psycopg2 #usada para se conectar e interagir com bancos de dados PostgreSQL
from datetime import datetime, timedelta #usados para trabalhar com datas e horas
import random #usada para gerar valores aleatórios

In [31]:
def conectar():
    return psycopg2.connect(
        dbname="ecommerce",
        user="postgres",
        password="gustavo1511",
        host="localhost",
        port="5432"
    )

In [32]:
# Gerando uma data aleatória
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

In [33]:
# Gerando um valor aleatório de preço
def random_price():
    return round(random.uniform(10.0, 1000.0), 2)

In [34]:
# Estabelece uma conexão com o banco de dados e cria um cursor para executar comandos SQL
conn = conectar()
cursor = conn.cursor()

# Remove todos os registros existentes numa tabela, garantindo que você comece com uma tabela vazia
cursor.execute("TRUNCATE TABLE Categoria RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE Clientes RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE Itens_pedido RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE Pagamentos RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE Pedidos RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE Produtos RESTART IDENTITY CASCADE;")
conn.commit()

try:
    # Inserção de dados na tabela Categoria
    for _ in range(10):
        nome = f"Categoria_{random.randint(1, 100)}"
        descricao = f"Descrição da {nome}"
        cursor.execute(
            "INSERT INTO Categoria (Nome, Descricao) VALUES (%s, %s) RETURNING ID_Categoria",
            (nome, descricao)
        )
        categoria_id = cursor.fetchone()[0]  # Armazena o ID da categoria inserida

    # Inserção de dados na tabela Produtos
    for _ in range(10000):
        nome = f"Produto_{random.randint(1, 10000)}"
        descricao = f"Descrição do {nome}"
        preco = random_price()
        estoque = random.randint(1, 100)
        categoria = random.randint(1, 10)  # Escolhe um ID de categoria existente
        cursor.execute(
            "INSERT INTO Produtos (Nome, Descricao, Preco, Estoque, Categoria) VALUES (%s, %s, %s, %s, %s)",
            (nome, descricao, preco, estoque, categoria)
        )

    # Inserção de dados na tabela Clientes
    clientes_ids = []
    for _ in range(10000):
        nome = f"Cliente_{random.randint(1, 10000)}"
        endereco = f"Endereço do {nome}"
        telefone = f"+5511{random.randint(100000000, 999999999)}"
        email = f"{nome}@exemplo.com"
        
        # Garantir que o email seja único
        while True:
            cursor.execute("SELECT COUNT(*) FROM Clientes WHERE Email = %s", (email,))
            if cursor.fetchone()[0] == 0:
                break
            email = f"{nome}{random.randint(1000, 9999)}@exemplo.com"  # Adiciona um número aleatório ao email
        
        data_nascimento = random_date(datetime(1950, 1, 1), datetime(2000, 12, 31)).date()
        cursor.execute(
            "INSERT INTO Clientes (Nome, Endereco, Telefone, Email, Data_Nascimento) VALUES (%s, %s, %s, %s, %s) RETURNING ID_Cliente",
            (nome, endereco, telefone, email, data_nascimento)
        )
        clientes_ids.append(cursor.fetchone()[0])  # Armazena o ID do cliente inserido

    # Inserção de dados na tabela Pedidos
    pedidos_ids = []
    for _ in range(10000):
        data_pedido = random_date(datetime(2020, 1, 1), datetime(2023, 12, 31))
        status = random.choice(['Pendente', 'Processando', 'Concluído', 'Cancelado'])
        valor_total = random_price()
        id_cliente = random.choice(clientes_ids)  # Escolhe um ID de cliente existente
        cursor.execute(
            "INSERT INTO Pedidos (Data_Pedido, Status, Valor_Total, ID_Cliente) VALUES (%s, %s, %s, %s) RETURNING ID_Pedido",
            (data_pedido, status, valor_total, id_cliente)
        )
        pedidos_ids.append(cursor.fetchone()[0])  # Armazena o ID do pedido inserido

    # Inserção de dados na tabela Itens_Pedido
    for _ in range(10000):
        quantidade = random.randint(1, 10)
        preco_unitario = random_price()
        subtotal = round(preco_unitario * quantidade, 2)
        id_pedido = random.choice(pedidos_ids)  # Escolhe um ID de pedido existente
        id_produto = random.randint(1, 10000)  # Supondo que o ID do produto está no intervalo de 1 a 10000
        cursor.execute(
            "INSERT INTO Itens_Pedido (Quantidade, Preco_Unitario, Subtotal, ID_Pedido, ID_Produto) VALUES (%s, %s, %s, %s, %s)",
            (quantidade, preco_unitario, subtotal, id_pedido, id_produto)
        )

    # Inserção de dados na tabela Pagamentos
    for _ in range(10000):
        data_pagamento = random_date(datetime(2020, 1, 1), datetime(2023, 12, 31)).date()
        valor_pago = random_price()
        metodo_pagamento = random.choice(['Cartão de Crédito', 'Boleto', 'Pix', 'PayPal'])
        id_pedido = random.choice(pedidos_ids)  # Escolhe um ID de pedido existente
        cursor.execute(
            "INSERT INTO Pagamentos (Data_Pagamento, Valor_Pago, Metodo_Pagamento, ID_Pedido) VALUES (%s, %s, %s, %s)",
            (data_pagamento, valor_pago, metodo_pagamento, id_pedido)
        )

    # Confirma as transações
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print(f"Erro ao inserir dados: {e}")
    conn.rollback()
finally:
    cursor.close()
    conn.close()

Dados inseridos com sucesso!
